## Imports

In [1]:
from SPARQLWrapper import SPARQLWrapper,JSON
import time
import pickle

In [2]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

In [3]:
from ipywidgets import IntProgress
from IPython.display import display

## Defines

In [4]:
def getQueryJson(query):
    sparql.setQuery(query)
    return sparql.query().convert()

In [5]:
def openMLfile(path):
    moviesML = open(path).readlines()
    for idx,line in enumerate(moviesML):
        moviesML[idx] = line.split("::")[1][:-7]
    return moviesML

In [6]:
def ensure_unicode(v):
    if isinstance(v, str):
        v = v.decode('latin-1')
    return v

In [7]:
def isInTDB(title):
    l = 0
    r = len(titlesDB)
    while(l <= r):
        mid = int((l+r)/2)
        if(ensure_unicode(titlesDB[mid]) == ensure_unicode(title)):
            return True
        if(ensure_unicode(titlesDB[mid]) < ensure_unicode(title)):
            l = mid+1
        else:
            r = mid-1
    return False

## Load data

Scrape all movies from dbpedia - 34 Queries ~340s

In [8]:
retrieved = True
moviesDB = pickle.load(open("pickles/moviesDB.pickle",'rb'))

In [9]:
if(moviesDB == []):
    retrieved = False
    a = [0 for i in range(10000)]
    c = 0
    beginTime = time.time()
    bar = IntProgress(min = 0,max = 34)
    display(bar)
    print "Query #0 of 34 - 0.000s elapsed\r",
    while len(a) >= 10000:
        bar.value = c
        yes = True
        while(yes):
            yes = False
            try:
                a = getQueryJson("""select distinct ?label,?film
                where {?film rdf:type <http://dbpedia.org/ontology/Film>.
                ?film rdfs:label ?label} LIMIT 10000 OFFSET """+str(c*10000))['results']['bindings']
            except KeyboardInterrupt:
                assert(False)
            except:
                yes = True
        moviesDB = moviesDB + a
        c += 1 
        print "Query #"+str(c)+" of 34 - %.3fs elapsed\r"%(time.time()-beginTime),
    print "\nCell executed in "+str(time.time()-beginTime)+"s"

In [10]:
if(not(retrieved)):
    pickle.dump(moviesDB,open("moviesDB.pickle",'w'))

In [11]:
print str(len(moviesDB))+" movies pulled from dbPedia"

342596 movies pulled from dbPedia


Extract titles from moviesDB

In [12]:
titlesDB = []
titlesLookup = {}
linksLookup = {}
for n in moviesDB:
    if (n['label']['xml:lang'] == 'en'):
        titlesDB += [n['label']['value'].split(" (")[0]]
        titlesLookup[n['label']['value'].split(" (")[0]] = n['film']['value']
        linksLookup[n['film']['value']] = n['label']['value'].split(" (")[0]
titlesDB.sort()
titlesDB[:10]

[u'!Women Art Revolution',
 u'"713" Requests Permission to Land',
 u'"Crocodile" Dundee',
 u'"Crocodile" Dundee II',
 u'"FF.SS." \u2013 Cio\xe8: "...che mi hai portato a fare sopra a Posillipo se non mi vuoi pi\xf9 bene?"',
 u'"Good Hair" and Other Dubious Distinctions',
 u'"Pimpernel" Smith',
 u'"Weird Al" Yankovic Live!',
 u'"Weird Al" Yankovic Live!: The Alpocalypse Tour',
 u'"Weird Al" Yankovic: The Ultimate Collection']

Read moviesML_20m from CSV

In [13]:
import csv
moviesML_20m = []
moviesML_20m_g = {}
moviesML_lookup = {}
index_lookup = {}
nums = []
with open("in/movies.csv") as csvfile:
    spamreader = csv.reader(csvfile, delimiter='~', quotechar='|')
    for row in spamreader:
        
        row = row[0]#.decode('latin-1')
        orig = row
        numId = row.split(',')[0]
        if(row[0] == 'm'):
            continue #skip header
        
        nums += [numId]
        
        while('0' <= row[0] and '9' >= row[0]):
            row = row[1:] #remove leading num ids
        
        row = row[1:] #remove first comma
        genres = row.split(',')[-1].split('|')
        
        if(row[0] == '\"'):
            row = row[1:].split('\"')[0]
            if(len(row) > 2):
                continue
        else:
            row = row.split(',')[0]
        row = row.replace(')','(').split('(')
        """if(len(row) == 1 or len(row[1]) == 0):
            print row
            print orig
            break"""
        if(len(row) > 1 and len(row[1]) > 0 and row[1][0] == 'a'):
            moviesML_20m += [row[1][7:]]
            moviesML_20m_g[row[1][7:]] = genres
            #moviesML_lookup[numId] = row[1][7:]
        moviesML_20m += [row[0][:-1]]
        moviesML_20m_g[row[0][:-1]] = genres
        moviesML_lookup[numId] = row[0][:-1]
len(moviesML_20m)
#print nums

21080

Cut blank titles

In [14]:
moviesML_20m.sort()
moviesML_20m = moviesML_20m[6:]

## Intersect MovieLens and DBpedia

In [15]:
intersect_20m = set(titlesDB).intersection(set(moviesML_20m))
len(intersect_20m)

15440

In [16]:
intersect_links = [[titlesLookup[x],x] for x in intersect_20m]
linksLookup = {}
for x in intersect_20m:
    linksLookup[titlesLookup[x]] = x
print len(intersect_links)
print len(linksLookup)

15440
15440


In [17]:
print "20M intersection percentage: "+str(100*len(intersect_20m)/float(len(moviesML_20m)))+"%"

20M intersection percentage: 73.2656353801%


In [18]:
print intersect_links[0]
moviesML_20m_g[linksLookup[intersect_links[0][0]]]

[u'http://dbpedia.org/resource/Eddie_Murphy_Delirious', 'Eddie Murphy Delirious']


['Comedy', 'Documentary']

# Query movie⟹person relations
### !! Very Slow !! ~3.2 hrs

In [19]:
retrieved = True
heads = pickle.load(open("pickles/heads.pickle","rb"))

In [20]:
if(heads == []):
    retrieved = False
    x = []
    cellStart = time.time()
    bar = IntProgress(min = 0, max = len(intersect_links))
    display(bar)
    for idx,entity in enumerate(intersect_links):
        bar.description = str(idx)+"/"+str(len(intersect_links))
        yes = True
        while(yes):
            yes = False
            try:
                x = getQueryJson("""SELECT ?r,?t,?tn WHERE
        {<"""+entity[0]+"""> ?r ?t.
        ?t rdfs:label ?tn.
        VALUES ?r
        {<http://dbpedia.org/ontology/director> <http://dbpedia.org/ontology/writer>
        <http://dbpedia.org/ontology/starring> <http://dbpedia.org/ontology/creator>
        <http://purl.org/dc/terms/subject> <http://dbpedia.org/ontology/musicComposer>
        <http://dbpedia.org/ontology/country> <http://dbpedia.org/property/story>
        <http://dbpedia.org/property/studio> <http://dbpedia.org/ontology/cinematography>
        <http://dbpedia.org/ontology/distributor>}.
        FILTER (LANG(?tn) = "en").}""")
            except KeyboardInterrupt:
                assert(False)
            except:
                yes = True
        heads += [[entity[0],x['results']['bindings']]]
        bar.value = idx+1
    print "-----"
    print "Execution done in "+str(time.time()-cellStart)+"s"

## Pickle heads file

In [21]:
if(not(retrieved)):
    heads.sort()
    pickle.dump(heads,open("heads.pickle",'wb'))

In [22]:
heads[:2]

[[u'http://dbpedia.org/resource/$9.99',
  [{u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
    u't': {u'type': u'uri',
     u'value': u'http://dbpedia.org/resource/Category:2008_films'},
    u'tn': {u'type': u'literal', u'value': u'2008 films', u'xml:lang': u'en'}},
   {u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
    u't': {u'type': u'uri',
     u'value': u'http://dbpedia.org/resource/Category:Israeli_films'},
    u'tn': {u'type': u'literal',
     u'value': u'Israeli films',
     u'xml:lang': u'en'}},
   {u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
    u't': {u'type': u'uri',
     u'value': u'http://dbpedia.org/resource/Category:Animated_drama_films'},
    u'tn': {u'type': u'literal',
     u'value': u'Animated drama films',
     u'xml:lang': u'en'}},
   {u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
    u't': {u'type': u'uri',
     u'value': u'http://dbpedia.org/resource/Category:

# Count entities

In [23]:
nonMovies = []
entities = []
entityLookup = {}
for headSet in heads:
    head = headSet[0].encode("utf-8")
    if(not(head in linksLookup)):
        continue
    lst = headSet[1]
    entityLookup[head] = len(entities)
    entities += [[head,linksLookup[head]]]
    for rel in lst:
        thing = rel['t']['value']
        name = rel['tn']['value']
        if(not(thing in entityLookup)):
            entityLookup[thing] = len(entities)
            entities += [[thing,name]]
            nonMovies += [[thing,name]]
print str(len(nonMovies))+" non-movie entities + "+str(len(intersect_links))+" movies = "+str(len(entities))+" total entities."

49908 non-movie entities + 15440 movies = 65347 total entities.


# Query person⟺person relations

In [24]:
retrieved = True
tails = pickle.load(open("pickles/tails.pickle",'rb'))

In [25]:
if (tails == []):
    retrieved = False
    x = []
    cellStart = time.time()
    bar = IntProgress(min = 0, max = len(nonMovies))
    display(bar)
    print "Entity 00000 of "+str(len(nonMovies))+", elapsed 0.00s -- ETA NaN\r",
    
    for idx,entity in enumerate(nonMovies):
        yes = True
        while(yes):
            yes = False
            try:
                x = getQueryJson("""SELECT ?r,?t,?tn WHERE
        {<"""+entity[0]+"""> ?r ?t.
        ?t rdfs:label ?tn.
        VALUES ?r
        {<http://dbpedia.org/ontology/spouse> <http://dbpedia.org/ontology/relative>}.
        FILTER (LANG(?tn) = "en").}""")
            except KeyboardInterrupt:
                assert(False)
            except:
                yes = True
        tails += [[entity[0],x['results']['bindings']]]
        print "Entity %05d of "%((idx+1))+str(len(nonMovies))+\
", elapsed %.2fs -- ETA %.2fs\r"\
%(time.time()-cellStart,(time.time()-cellStart)*((len(nonMovies)-idx-1)/(idx+1))),
        bar.value = idx+1
    print "\n----------------------------------------"
    print "Execution done in %.2fs"%(time.time()-cellStart)

In [26]:
if(not(retrieved)):
    pickle.dump(tails,open("pickles/tails.pickle","wb"))

## Add genres into heads

In [27]:
for idx,headSet in enumerate(heads):
    head = headSet[0]
    #print head
    try:
        for genre in moviesML_20m_g[linksLookup[head].encode('utf-8')]:
            heads[idx][1] += [{'r': {'value': 'http://dbpedia.org/ontology/genre'},
                               't': {'value': genre},
                               'tn': {'value': genre}}]
    except:
        continue
heads[1]

[u'http://dbpedia.org/resource/$_(film)',
 [{u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
   u't': {u'type': u'uri',
    u'value': u'http://dbpedia.org/resource/Category:1970s_crime_films'},
   u'tn': {u'type': u'literal',
    u'value': u'1970s crime films',
    u'xml:lang': u'en'}},
  {u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
   u't': {u'type': u'uri',
    u'value': u'http://dbpedia.org/resource/Category:Heist_films'},
   u'tn': {u'type': u'literal', u'value': u'Heist films', u'xml:lang': u'en'}},
  {u'r': {u'type': u'uri', u'value': u'http://dbpedia.org/ontology/starring'},
   u't': {u'type': u'uri',
    u'value': u'http://dbpedia.org/resource/Goldie_Hawn'},
   u'tn': {u'type': u'literal', u'value': u'Goldie Hawn', u'xml:lang': u'en'}},
  {u'r': {u'type': u'uri', u'value': u'http://purl.org/dc/terms/subject'},
   u't': {u'type': u'uri',
    u'value': u'http://dbpedia.org/resource/Category:Films_directed_by_Richard_Brooks'},
  

In [28]:
def relationTailType(num):
    if(num%2 == 1):
        if(num == 25):
            return 'Person'
        return 'Movie'
    if(num == 8):
        return 'Category'
    if(num == 12):
        return 'Country'
    if(num == 16 or num == 22):
        return 'Company'
    if(num == 18):
        return 'Genre'
    return 'Person'

In [29]:
relationsIdx = {"http://dbpedia.org/ontology/director":0,
                "http://dbpedia.org/ontology/writer":2,
                "http://dbpedia.org/ontology/starring":4,
                "http://dbpedia.org/ontology/creator":6,
                "http://purl.org/dc/terms/subject":8, #http://purl.org/dc/terms/subject
                "http://dbpedia.org/ontology/musicComposer":10,
                "http://dbpedia.org/ontology/country":12,
                "http://dbpedia.org/property/story":14, #http://dbpedia.org/property/story
                "http://dbpedia.org/property/studio":16, #http://dbpedia.org/property/studio
                "http://dbpedia.org/ontology/genre":18,
                "http://dbpedia.org/ontology/cinematography":20,
                "http://dbpedia.org/ontology/distributor":22,
                "http://dbpedia.org/ontology/spouse":24,
                "http://dbpedia.org/ontology/relative":25}
idxRelations = {0:"directedBy", 1:"directorOf",
                2:"writtenBy",   3:"writerOf",
                4:"starring", 5:"starredIn",
                6:"createdBy",  7:"creatorOf",
                8:"subject",  9:"subjectOf",
                10:"musicComposedBy", 11:"musicComposerOf",
                12:"country", 13:"countryOf",
                14:"storyBy",   15:"storywriterOf",
                16:"studio",  17:"studioOf",
                18:"genre",   19:"genreOf",
                20:"cinematographyBy", 21:"cinematographerOf",
                22:"distributedBy", 23:"distributorOf",
                24:"spouse",
                25:"relative"}
for a in relationsIdx:
    print relationTailType(relationsIdx[a]+1)+" - "+a+" - "+relationTailType(relationsIdx[a])

Movie - http://dbpedia.org/ontology/genre - Genre
Movie - http://purl.org/dc/terms/subject - Category
Movie - http://dbpedia.org/ontology/cinematography - Person
Movie - http://dbpedia.org/ontology/musicComposer - Person
Movie - http://dbpedia.org/property/story - Person
Person - http://dbpedia.org/ontology/spouse - Person
Movie - http://dbpedia.org/ontology/distributor - Company
Movie - http://dbpedia.org/ontology/director - Person
Movie - http://dbpedia.org/ontology/country - Country
Movie - http://dbpedia.org/property/studio - Company
Movie - http://dbpedia.org/ontology/starring - Person
Movie - http://dbpedia.org/ontology/writer - Person
Person - http://dbpedia.org/ontology/relative - Person
Movie - http://dbpedia.org/ontology/creator - Person


## EntityType has been forgotten for tails in the cell below

In [65]:
file = open("out/triples_idx2.txt",'w')
triples_idx = []


entitiesIdx = {}
moviesIdx = {}
entities = []
movies = []
entityType = {}

for a in heads:
    if(not(a[0] in linksLookup)):
        continue
    if(len(a[1]) > 0):
        moviesIdx[a[0]] = len(movies)
        movies += [a[0]]
        
    entityType[a[0]] = relationTailType(relationsIdx[a[1][0]['r']['value']]+1)
        
    for b in a[1]:
        entity = b["t"]["value"]
        entityName = b["tn"]["value"]
        relation = b["r"]["value"]
        
        entityType[entity] = relationTailType(relationsIdx[relation])
        
        if(not(entity in entitiesIdx)):
            entitiesIdx[entity] = len(entities)+len(intersect_links)
            entities += [entity]
        
        rIdx = relationsIdx[relation]
        revIdx = rIdx+1
        
        triples_idx += [[moviesIdx[a[0]],rIdx,entitiesIdx[entity]]]
        triples_idx += [[entitiesIdx[entity],revIdx,moviesIdx[a[0]]]]
        file.write(str(moviesIdx[a[0]])+" "+str(200000+rIdx)+" "+str(entitiesIdx[entity])+"\n")
        file.write(str(entitiesIdx[entity])+" "+str(200000+revIdx)+" "+str(moviesIdx[a[0]])+"\n")
        
for a in tails:
    if(not(a[0] in entitiesIdx)):
        continue
    for b in a[1]:
        entity = b["t"]["value"]
        entityName = b["tn"]["value"]
        relation = b["r"]["value"]
        
        if(not(entity in entitiesIdx)):
            entitiesIdx[entity] = len(entities)+len(intersect_links)
            entities += [entity]
        
        rIdx = relationsIdx[relation]
        revIdx = rIdx
        
        triples_idx += [[entitiesIdx[a[0]],rIdx,entitiesIdx[entity]]]
        triples_idx += [[entitiesIdx[entity],revIdx,entitiesIdx[a[0]]]]
        file.write(str(entitiesIdx[a[0]])+" "+str(200000+rIdx)+" "+str(entitiesIdx[entity])+"\n")
        file.write(str(entitiesIdx[entity])+" "+str(200000+revIdx)+" "+str(entitiesIdx[a[0]])+"\n")
        
file.close()

In [66]:
numIdLookup = {}

for nId in nums:
    try:
        #print "\t\tNEW"
        ttl = moviesML_lookup[nId]
        #print ttl
        uri = titlesLookup[ttl]
        #print uri
        idx = moviesIdx[uri]
        #print idx
        numIdLookup[nId] = idx
    except KeyError:
        #print "KeyError"
        continue
    
numIdLookup
#maps original -> filtered.

{'73469': 9097,
 '73399': 7734,
 '3382': 12247,
 '110553': 13197,
 '98160': 9374,
 '110551': 1933,
 '82102': 11703,
 '73392': 2990,
 '73390': 10576,
 '70175': 15093,
 '89371': 10792,
 '89373': 2592,
 '101763': 9179,
 '89375': 12908,
 '101761': 14782,
 '38843': 195,
 '130462': 13224,
 '116660': 5128,
 '5988': 10740,
 '5989': 2551,
 '55294': 14770,
 '5982': 4456,
 '5983': 6925,
 '5980': 1704,
 '5986': 4694,
 '5985': 1009,
 '82449': 6596,
 '103541': 13157,
 '96075': 1787,
 '87960': 1554,
 '82194': 14034,
 '103547': 14539,
 '48895': 8023,
 '120438': 5347,
 '73462': 4432,
 '130642': 1173,
 '120432': 278,
 '43871': 4853,
 '120126': 1388,
 '73464': 3847,
 '128870': 15273,
 '122015': 11664,
 '101498': 6792,
 '128878': 88,
 '55343': 12618,
 '111653': 9519,
 '57484': 160,
 '27370': 7753,
 '6797': 2241,
 '6794': 1417,
 '6795': 1728,
 '6793': 1421,
 '6790': 1099,
 '6791': 1130,
 '27378': 8093,
 '6798': 2242,
 '6799': 2314,
 '61941': 6481,
 '97153': 9639,
 '98643': 7549,
 '75947': 8918,
 '99999': 1

In [67]:
import csv
#moviesML_20m = []
#moviesML_20m_g = {}
#moviesML_lookup = {}

file = open("out/movies_re.csv","w")

with open("in/movies.csv") as csvfile:
    csvreader = csv.reader(csvfile, delimiter='~', quotechar='|')
    
    for row in csvreader:
        
        row = row[0]#.decode('latin-1')
        orig = row
        numId = row.split(',')[0]
        if(row[0] == 'm'):
            file.write(orig+"\n")
            continue #skip header
            
        while('0' <= row[0] and '9' >= row[0]):
            row = row[1:] #remove leading num ids
        
        if(numId in numIdLookup):
            file.write(str(numIdLookup[numId])+row+"\n");
file.close()

In [68]:
file = open("out/IdxToEID.txt","w")

for idx in numIdLookup:
    file.write(str(idx)+" "+str(numIdLookup[idx])+"\n")

file.close()

print len(numIdLookup)

16408


In [74]:
file = open("out/moviesIdx.txt",'w')

for uri in movies:
    idx = moviesIdx[uri]
    file.write(uri.encode('utf-8')+" "+str(idx)+"\n")
file.close()

In [72]:
file = open("out/entities.txt","w")

for idx,entity in enumerate(entities):
    #print entity
    file.write(entity.encode('utf-8')+" "+str(idx+len(intersect_links))+"\n")
    
file.close()

In [73]:
file = open("out/types.txt","w")

for idx in entityType:
    typ = entityType[idx]
    file.write(idx.encode('utf-8')+"\t"+typ+"\n")
file.close()

In [83]:
lst = ['Person','Movie','Category','Country','Company','Genre']

for typ in lst:
    file = open("out/perType/"+typ+".txt","w")

    for idx in entityType:
        if(typ == entityType[idx]):
            if(typ == 'Movie'):
                num = moviesIdx[idx]
            else:
                num = entitiesIdx[idx]
            file.write(idx.encode('utf-8')+" "+str(num)+"\n")
    file.close()

In [90]:
for relNum in idxRelations:
    rel = idxRelations[relNum]
    file = open("out/perRel/"+rel+".txt","w")
    for trpl in triples_idx:
        if(trpl[1] == relNum):
            file.write(str(trpl[0])+","+str(trpl[2])+"\n")
    file.close()
    print(rel+" done")

directedBy done
directorOf done
writtenBy done
writerOf done
starring done
starredIn done
createdBy done
creatorOf done
subject done
subjectOf done
musicComposedBy done
musicComposerOf done
country done
countryOf done
storyBy done
storywriterOf done
studio done
studioOf done
genre done
genreOf done
cinematographyBy done
cinematographerOf done
distributedBy done
distributorOf done
spouse done
relative done
